## data


In [ ]:
!pip install azure-storage-blob azure-ai-formrecognizer -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.5 MB/s eta 0:00:00


In [ ]:
AZURE_FORM_RECOGNIZER_ENDPOINT = "https://agentic-pod-document-intelligence-svc.cognitiveservices.azure.com/"
AZURE_FORM_RECOGNIZER_KEY = "5xdSJ4ijR0LlX17Cop9qZYhGhwjsWEIbUjPiSv0CBFzVgvNjFMyEJQQJ99BDAC5RqLJXJ3w3AAALACOGTyLg"

AZURE_BLOB_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=agenticpodstorageacnt;AccountKey=WbE1Lm0JH1aTSuBWw7Lhlrc0igamXOIqOcOtePDrL3gDhWWfCE6ul4jDw2tdVE09KG56HxhbEDHc+AStBHv5dQ==;EndpointSuffix=core.windows.net"
AZURE_BLOB_CONTAINER_NAME = "ocr-docs"
AZURE_BLOB_PDF_NAME = "SampleContract-Shuttle.pdf"

In [ ]:
import os
import tempfile
import json
from azure.storage.blob import BlobServiceClient
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

def download_pdf_from_blob():
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_BLOB_CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container=AZURE_BLOB_CONTAINER_NAME, blob=AZURE_BLOB_PDF_NAME)

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    with open(temp_file.name, "wb") as f:
        f.write(blob_client.download_blob().readall())

    print(f"Downloaded PDF to {temp_file.name}")
    return temp_file.name

def analyze_pdf_by_page(pdf_path):
    document_analysis_client = DocumentAnalysisClient(
        endpoint=AZURE_FORM_RECOGNIZER_ENDPOINT,
        credential=AzureKeyCredential(AZURE_FORM_RECOGNIZER_KEY)
    )

    results = {}
    with open(pdf_path, "rb") as f:
        poller = document_analysis_client.begin_analyze_document("prebuilt-document", document=f)
        result = poller.result()

        for page in doc_result.pages:
            page_info = {
                "page_number": page.page_number,
                "width": page.width,
                "height": page.height,
                "unit": page.unit,
                "lines": [],
                "words": []
            }
            for line in page.lines:
                page_info["lines"].append({
                    "content": line.content,
                    "bounding_box": [p for p in line.polygon]
                })

            for word in page.words:
                page_info["words"].append({
                    "content": word.content,
                    "confidence": word.confidence
                })
        for page in result.pages:
            page_data = {
                "page_number": page.page_number,
                "width": page.width,
                "height": page.height,
                "unit": page.unit,
                "lines": []
            }
            for line in page.lines:
                page_data["lines"].append({
                    "content": line.content,
                    "bounding_box": [p for p in line.polygon]
                })
            results[f"page_{page.page_number}"] = page_data
    return results

if __name__ == "__main__":
    pdf_path = download_pdf_from_blob()
    json_output = analyze_pdf_by_page(pdf_path)

    print(json.dumps(json_output, indent=2))


Streaming output truncated to the last 5000 lines.
      },
      {
        "content": "more than fifteen (15) employees, provided that the foregoing provisions shall not apply to contracts",
        "bounding_box": [
          [
            1.4851,
            4.316
          ],
          [
            7.6404,
            4.3207
          ],
          [
            7.6404,
            4.4878
          ],
          [
            1.4851,
            4.4831
          ]
        ]
      },
      {
        "content": "or subcontracts for standard commercial supplies or raw materials.",
        "bounding_box": [
          [
            1.4803,
            4.4974
          ],
          [
            5.5632,
            4.4974
          ],
          [
            5.5632,
            4.6597
          ],
          [
            1.4803,
            4.6549
          ]
        ]
      },
      {
        "content": "7. FEDERAL, STATE AND LOCAL LAWS. CONSULTANT warrants that in the performance of thi

# **From Azure Documentation -->**

In [ ]:
# import libraries
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://agentic-pod-document-intelligence-svc.cognitiveservices.azure.com/"
key = "5xdSJ4ijR0LlX17Cop9qZYhGhwjsWEIbUjPiSv0CBFzVgvNjFMyEJQQJ99BDAC5RqLJXJ3w3AAALACOGTyLg"

def format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in polygon])


def analyze_layout():
    # sample document
    formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    poller = document_analysis_client.begin_analyze_document_from_url(
        "prebuilt-layout", formUrl
    )
    result = poller.result()

    for idx, style in enumerate(result.styles):
        print(
            "Document contains {} content".format(
                "handwritten" if style.is_handwritten else "no handwritten"
            )
        )

    for page in result.pages:
        print("----Analyzing layout from page #{}----".format(page.page_number))
        print(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            words = line.get_words()
            print(
                "...Line # {} has word count {} and text '{}' within bounding polygon '{}'".format(
                    line_idx,
                    len(words),
                    line.content,
                    format_polygon(line.polygon),
                )
            )

            for word in words:
                print(
                    "......Word '{}' has a confidence of {}".format(
                        word.content, word.confidence
                    )
                )

        for selection_mark in page.selection_marks:
            print(
                "...Selection mark is '{}' within bounding polygon '{}' and has a confidence of {}".format(
                    selection_mark.state,
                    format_polygon(selection_mark.polygon),
                    selection_mark.confidence,
                )
            )

    for table_idx, table in enumerate(result.tables):
        print(
            "Table # {} has {} rows and {} columns".format(
                table_idx, table.row_count, table.column_count
            )
        )
        for region in table.bounding_regions:
            print(
                "Table # {} location on page: {} is {}".format(
                    table_idx,
                    region.page_number,
                    format_polygon(region.polygon),
                )
            )
        for cell in table.cells:
            print(
                "...Cell[{}][{}] has content '{}'".format(
                    cell.row_index,
                    cell.column_index,
                    cell.content,
                )
            )
            for region in cell.bounding_regions:
                print(
                    "...content on page {} is within bounding polygon '{}'".format(
                        region.page_number,
                        format_polygon(region.polygon),
                    )
                )

    print("----------------------------------------")


if __name__ == "__main__":
    analyze_layout()

Document contains handwritten content
----Analyzing layout from page #1----
Page has width: 8.5 and height: 11.0, measured with unit: inch
...Line # 0 has word count 2 and text 'UNITED STATES' within bounding polygon '[3.4669, 0.6636], [5.0236, 0.6589], [5.0236, 0.8451], [3.4669, 0.8498]'
......Word 'UNITED' has a confidence of 0.995
......Word 'STATES' has a confidence of 0.992
...Line # 1 has word count 4 and text 'SECURITIES AND EXCHANGE COMMISSION' within bounding polygon '[2.168, 0.8737], [6.3129, 0.8737], [6.3129, 1.0647], [2.168, 1.0647]'
......Word 'SECURITIES' has a confidence of 0.994
......Word 'AND' has a confidence of 0.998
......Word 'EXCHANGE' has a confidence of 0.995
......Word 'COMMISSION' has a confidence of 0.993
...Line # 2 has word count 3 and text 'Washington, D.C. 20549' within bounding polygon '[3.443, 1.0885], [5.057, 1.0838], [5.057, 1.2509], [3.443, 1.2604]'
......Word 'Washington,' has a confidence of 0.99
......Word 'D.C.' has a confidence of 0.989
......W

## translation


In [2]:
import os
import requests
import uuid
import json

# Azure configuration
AZURE_TRANSLATOR_ENDPOINT = os.getenv("AZURE_TRANSLATOR_ENDPOINT", "https://api.cognitive.microsofttranslator.com")
AZURE_TRANSLATOR_KEY = os.getenv("AZURE_TRANSLATOR_KEY", "Ep3RIqbUBCpskWV2YnwcRtaR5PTMPHS3zKbTjOZouwonzSjeyCVMJQQJ99BEAC5RqLJXJ3w3AAAbACOGa5uN ")  # Replace with your actual Azure key
AZURE_TRANSLATOR_REGION = os.getenv("AZURE_TRANSLATOR_REGION", "westeurope")   # Replace if needed, e.g., "eastus"

# File paths
INPUT_FILE = "/content/English.txt"
OUTPUT_FILE = "output_fr.txt"

def translate_text(text, from_lang="en", to_lang="fr"):
    url = f"{AZURE_TRANSLATOR_ENDPOINT}/translate?api-version=3.0&from={from_lang}&to={to_lang}"

    headers = {
        "Ocp-Apim-Subscription-Key": AZURE_TRANSLATOR_KEY,
        "Ocp-Apim-Subscription-Region": AZURE_TRANSLATOR_REGION,
        "Content-Type": "application/json",
        "X-ClientTraceId": str(uuid.uuid4())
    }

    body = [{"text": text}]
    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()
    return response.json()[0]['translations'][0]['text']

def translate_file(input_path, output_path, target_language="fr"):
    with open(input_path, "r", encoding="utf-8") as f:
        text = f.read()

    translated_text = translate_text(text, from_lang="en", to_lang=target_language)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(translated_text)

    print(f"Translated '{input_path}' → '{output_path}' ({target_language})")

if __name__ == "__main__":
    translate_file(INPUT_FILE, OUTPUT_FILE, target_language="fr")


Translated '/content/English.txt' → 'output_fr.txt' (fr)


## text

In [18]:
pip install azure-storage-blob requests python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 18.1 MB/s eta 0:00:00


In [3]:
import os
import requests
import uuid
import json
from azure.storage.blob import BlobServiceClient

# Azure Translator configuration
AZURE_TRANSLATOR_ENDPOINT = os.getenv("AZURE_TRANSLATOR_ENDPOINT", "https://api.cognitive.microsofttranslator.com")
AZURE_TRANSLATOR_KEY = os.getenv("AZURE_TRANSLATOR_KEY", "Ep3RIqbUBCpskWV2YnwcRtaR5PTMPHS3zKbTjOZouwonzSjeyCVMJQQJ99BEAC5RqLJXJ3w3AAAbACOGa5uN ")  # Replace with your actual Azure key
AZURE_TRANSLATOR_REGION = os.getenv("AZURE_TRANSLATOR_REGION", "westeurope")

# Azure Blob Storage configuration
AZURE_BLOB_CONNECTION_STRING = os.getenv("AZURE_BLOB_CONNECTION_STRING", "DefaultEndpointsProtocol=https;AccountName=agenticpodstorageacnt;AccountKey=WbE1Lm0JH1aTSuBWw7Lhlrc0igamXOIqOcOtePDrL3gDhWWfCE6ul4jDw2tdVE09KG56HxhbEDHc+AStBHv5dQ==;EndpointSuffix=core.windows.net")
INPUT_CONTAINER_NAME = "translation-input"
INPUT_BLOB_NAME = "English.txt"
OUTPUT_CONTAINER_NAME = "translation-input"
OUTPUT_BLOB_NAME = "Translated_French.txt"

def translate_text(text, from_lang="en", to_lang="fr"):
    url = f"{AZURE_TRANSLATOR_ENDPOINT}/translate?api-version=3.0&from={from_lang}&to={to_lang}"

    headers = {
        "Ocp-Apim-Subscription-Key": AZURE_TRANSLATOR_KEY,
        "Ocp-Apim-Subscription-Region": AZURE_TRANSLATOR_REGION,
        "Content-Type": "application/json",
        "X-ClientTraceId": str(uuid.uuid4())
    }

    body = [{"text": text}]
    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()
    return response.json()[0]['translations'][0]['text']

def read_blob(blob_service_client, container_name, blob_name):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_data = blob_client.download_blob()
    return blob_data.readall().decode("utf-8")

def write_blob(blob_service_client, container_name, blob_name, content):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(content, overwrite=True)

def main():
    # Initialize Blob service client
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_BLOB_CONNECTION_STRING)

    # Step 1: Read input text from Blob
    input_text = read_blob(blob_service_client, INPUT_CONTAINER_NAME, INPUT_BLOB_NAME)

    # Step 2: Translate text
    translated_text = translate_text(input_text, from_lang="en", to_lang="fr")

    # Step 3: Write translated text back to Blob
    write_blob(blob_service_client, OUTPUT_CONTAINER_NAME, OUTPUT_BLOB_NAME, translated_text)

    print(f"Translated file saved to '{OUTPUT_CONTAINER_NAME}/{OUTPUT_BLOB_NAME}'")

if __name__ == "__main__":
    main()


Translated file saved to 'translation-input/Translated_French.txt'


## upload metadeta

In [ ]:
from azure.storage.blob import BlobServiceClient, ContentSettings

# Config
CONNECTION_STRING = ZURE_BLOB_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=agenticpodstorageacnt;AccountKey=WbE1Lm0JH1aTSuBWw7Lhlrc0igamXOIqOcOtePDrL3gDhWWfCE6ul4jDw2tdVE09KG56HxhbEDHc+AStBHv5dQ==;EndpointSuffix=core.windows.net")
CONTAINER_NAME = "translation-input"
BLOB_NAME = "glossary_terms.txt"
USERNAME = "john_doe"

# Metadata
metadata = {
    "username": USERNAME,
    "glossary_type": "industry_terms",
    "entity_focus": "manufacturing",
    "ner_mode": "enhanced",
    "upload_purpose": "custom_ner_training",
    "upload_timestamp": "2025-05-14T10:00:00Z"
}

# Upload the file
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=BLOB_NAME)

with open("path/to/glossary_terms.txt", "rb") as data:
    blob_client.upload_blob(
        data,
        overwrite=True,
        content_settings=ContentSettings(content_type="text/plain"),
        metadata=metadata
    )


In [19]:
from azure.storage.blob import BlobServiceClient, ContentSettings
import os

CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=agenticpodstorageacnt;AccountKey=WbE1Lm0JH1aTSuBWw7Lhlrc0igamXOIqOcOtePDrL3gDhWWfCE6ul4jDw2tdVE09KG56HxhbEDHc+AStBHv5dQ==;EndpointSuffix=core.windows.net"

CONTAINER_NAME = "translation-input"

# 📂 Your container and file details
FILE_PATH = "/content/metadata_trans.txt"  # Change this to your actual file path
USERNAME = "john_doe"
BLOB_NAME = os.path.basename(FILE_PATH)

# 🔧 Initialize blob client
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=BLOB_NAME)

# 📤 Upload file using your exact snippet
with open(FILE_PATH, "rb") as data:
    blob_client.upload_blob(
        data,
        overwrite=True,
        content_settings=ContentSettings(content_type="application/pdf"),
        metadata={
            "username": USERNAME,
            "filename": BLOB_NAME
        }
    )

print(f"✅ File '{BLOB_NAME}' uploaded successfully with metadata.")


✅ File 'metadata_trans.txt' uploaded successfully with metadata.
